# Vive Pro Eye Evaluation - Linear Model for Vision Correction x HMD

This notebook reproduces the following figures and results from the manuscript: 

- Linear model results for the factors vision correction x HMD, applied to accuracy and SD precision
- Figure 6: LMM results / estimated marginal means

Running this notebook requires the `rpy2` Python package and a working R installation. Model statistics are reported directly in the R output, while estimated marginal means are extracted and plotted in Figure 6. Requires the following R packages: `lme4`, `sjstats`, `pwr`, `emmeans`

In [1]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy.stats import linregress

from analysis import * 

FIGTYPES = ['png']


Note: vexptoolbox is not running under Vizard, or Vizard packages could not be imported. Only analysis tools will be available.


In [2]:
# Folders
folder_pkl = '.'
folder_results = '../results'

# Load preprocessed data
(tar, tar_i10, val, pp, sam) = load_pickle_data(folder_pkl)


Data loaded from pickles.


## Linear Model for Accuracy (R)

In [ ]:
# Initialize rpy2 extension for Jupyter (enables R code cells)
%load_ext rpy2.ipython

# rpy2 on Windows likes to throw error pop-ups about utf-8 decoding, suppress these here
from rpy2.rinterface_lib.callbacks import logger as rpy2_logger
import logging
rpy2_logger.setLevel(logging.CRITICAL)


In [ ]:
%%R
print(R.version$version)


In [ ]:
%%R -i val -o em_acc -o model_acc

library('lme4')
library('emmeans')
library('sjstats')

formula <- acc_nomonoc ~ 1 + vision + hmd + vision:hmd
model_acc <- lm(data=val, formula)

# Model + Anova results
cat('* ANOVA Omnibus Results + Statistics: Accuracy')
print(summary(model_acc))
print(anova_stats(model_acc))

# Post-Hoc Tests
cat('\n\n* Post-Hoc Comparisons: Accuracy')
print(emmeans(model_acc, pairwise ~ vision, adjust="holm")$contrasts)
print(emmeans(model_acc, pairwise ~ hmd, adjust="holm")$contrasts)

# Export estimated marginal means (for Figure 6)
em_acc <- summary(emmeans(model_acc, specs = ~ 'vision * hmd'))


## Linear Model for SD Precision (R)

In [ ]:
%%R -i val -o em_sd -o model_sd

library('lme4')
library('emmeans')
library('sjstats')

formula <- sd_nomonoc ~ 1 + vision + hmd + vision:hmd
model_sd <- lm(data=val, formula)

# Model + Anova results
cat('* ANOVA Omnibus Results + Statistics: SD Precision')
print(summary(model_sd))
print(anova_stats(model_sd))

# Post-Hoc Tests
cat('\n\n* Post-Hoc Comparisons: SD Precision')
print(emmeans(model_sd, pairwise ~ vision, adjust="holm")$contrasts)
print(emmeans(model_sd, pairwise ~ hmd, adjust="holm")$contrasts)

# Export estimated marginal means (for Figure 6)
em_sd <- summary(emmeans(model_sd, specs = ~ 'vision * hmd'))


In [ ]:
# Estimated Marginal Means - Accuracy
round(em_acc, 3)

In [ ]:
# Estimated Marginal Means - SD Precision
round(em_sd, 3)

## Figure 6: Linear Model results

In [ ]:
# Restructure the R results into the same dict structure used to plot
emmeans_acc = {'HMD 1': {'mean': [em_acc.loc[(em_acc.vision == 'uncorrected') & (em_acc.hmd == 1), 'emmean'].values[0],
                            em_acc.loc[(em_acc.vision == 'contacts') & (em_acc.hmd == 1), 'emmean'].values[0],
                            em_acc.loc[(em_acc.vision == 'glasses') & (em_acc.hmd == 1), 'emmean'].values[0]], 
                         'sem':  [em_acc.loc[(em_acc.vision == 'uncorrected') & (em_acc.hmd == 1), 'SE'].values[0],
                            em_acc.loc[(em_acc.vision == 'contacts') & (em_acc.hmd == 1), 'SE'].values[0],
                            em_acc.loc[(em_acc.vision == 'glasses') & (em_acc.hmd == 1), 'SE'].values[0]]},
               'HMD 2': {'mean': [em_acc.loc[(em_acc.vision == 'uncorrected') & (em_acc.hmd == 2), 'emmean'].values[0],
                            em_acc.loc[(em_acc.vision == 'contacts') & (em_acc.hmd == 2), 'emmean'].values[0],
                            em_acc.loc[(em_acc.vision == 'glasses') & (em_acc.hmd == 2), 'emmean'].values[0]], 
                         'sem':  [em_acc.loc[(em_acc.vision == 'uncorrected') & (em_acc.hmd == 2), 'SE'].values[0],
                            em_acc.loc[(em_acc.vision == 'contacts') & (em_acc.hmd == 2), 'SE'].values[0],
                            em_acc.loc[(em_acc.vision == 'glasses') & (em_acc.hmd == 2), 'SE'].values[0]]} }

emmeans_std = {'HMD 1': {'mean': [em_sd.loc[(em_sd.vision == 'uncorrected') & (em_sd.hmd == 1), 'emmean'].values[0],
                            em_sd.loc[(em_sd.vision == 'contacts') & (em_sd.hmd == 1), 'emmean'].values[0],
                            em_sd.loc[(em_sd.vision == 'glasses') & (em_sd.hmd == 1), 'emmean'].values[0]], 
                         'sem':  [em_sd.loc[(em_sd.vision == 'uncorrected') & (em_sd.hmd == 1), 'SE'].values[0],
                            em_sd.loc[(em_sd.vision == 'contacts') & (em_sd.hmd == 1), 'SE'].values[0],
                            em_sd.loc[(em_sd.vision == 'glasses') & (em_sd.hmd == 1), 'SE'].values[0]]},
               'HMD 2': {'mean': [em_sd.loc[(em_sd.vision == 'uncorrected') & (em_sd.hmd == 2), 'emmean'].values[0],
                            em_sd.loc[(em_sd.vision == 'contacts') & (em_sd.hmd == 2), 'emmean'].values[0],
                            em_sd.loc[(em_sd.vision == 'glasses') & (em_sd.hmd == 2), 'emmean'].values[0]], 
                         'sem':  [em_sd.loc[(em_sd.vision == 'uncorrected') & (em_sd.hmd == 2), 'SE'].values[0],
                            em_sd.loc[(em_sd.vision == 'contacts') & (em_sd.hmd == 2), 'SE'].values[0],
                            em_sd.loc[(em_sd.vision == 'glasses') & (em_sd.hmd == 2), 'SE'].values[0]]} }

In [ ]:
def lmm_accuracy_figure(data, figsize=(3.25, 3.2), dodge=0.09):
    
    VISION = ['No Correction', 'Contacts', 'Glasses']
    LABEL_SIZE = 10
    TICK_SIZE = 8
    MARKER_SIZE = 4
    ERRBAR_WIDTH = 1.2
    LINE_WIDTH = 0.8
    HMD_COLORS = [Set1_9.mpl_colors[1], Set1_9.mpl_colors[4], (0.5, 0.5, 0.5)]
    
    fig, ax = plt.subplots(1, 1, figsize=figsize, dpi=150)

    handles = []
    labels = []
    for h_ix, hmd in enumerate(data[0].keys()):
        xs = []
        ys_acc = []
        ys_std = []
        for vx, vision in enumerate(VISION):
            c = HMD_COLORS[h_ix]
            if hmd == 'HMD 1':
                x = vx - dodge + 1
            elif hmd == 'HMD 2':
                x = vx + dodge + 1
            else:
                x = vx + 1
            
            # Accuracy
            h1 = ax.errorbar(x, 
                             data[0][hmd]['mean'][vx], 
                             yerr=data[0][hmd]['sem'][vx], 
                             marker='o', 
                             color=c, 
                             linewidth=LINE_WIDTH, 
                             ms=MARKER_SIZE, 
                             zorder=3, 
                             label=hmd,
                            elinewidth=ERRBAR_WIDTH)
            ys_acc.append(data[0][hmd]['mean'][vx])
            
            # SD
            h2 = ax.errorbar(x, 
                             data[1][hmd]['mean'][vx], 
                             yerr=data[1][hmd]['sem'][vx], 
                             marker='o', 
                             color=c, 
                             markerfacecolor='w', 
                             linewidth=LINE_WIDTH, 
                             ms=MARKER_SIZE, 
                             zorder=3, 
                             label=hmd,
                            elinewidth=ERRBAR_WIDTH)
            ys_std.append(data[1][hmd]['mean'][vx])
            xs.append(x)
            
            if vx == 0:
                handles.append(h1)
                labels.append(hmd)
                            
        ax.plot(xs, ys_acc, '-', color=c, linewidth=LINE_WIDTH, zorder=2)
        ax.plot(xs, ys_std, ':', color=c, linewidth=LINE_WIDTH, zorder=2)

    ax.set_xlim([0.5, 3.5])
    ax.set_xticks(np.arange(1, len(VISION) + 1))
    ax.set_xticklabels(VISION, fontsize=TICK_SIZE)
    
    ax.set_xlabel('Vision Correction', fontsize=LABEL_SIZE) 
    ax.set_ylabel('Estimated Marginal Means (°)', fontsize=LABEL_SIZE)
    ax.set_ylim([0, 2])
    ax.set_yticks(np.arange(0, 2.5, 0.5))
    ax.tick_params(labelsize=TICK_SIZE, width=1.0, length=4, direction='in')
    
    # Combined Legend
    measures = [plt.Line2D([0], [0], color='k', marker='o', ls='-', linewidth=LINE_WIDTH, ms=MARKER_SIZE, label='Accuracy (Gaze Error)'),
                plt.Line2D([0], [0], color='k', marker='o', ls=':', markerfacecolor='w', linewidth=LINE_WIDTH, ms=MARKER_SIZE, label='Precision (SD)')]
    if len(data[0].keys()) > 1:
        measures.extend(handles)
    leg = ax.legend(handles=measures,
                    loc='upper left', 
                    fontsize=7, 
                    facecolor='w', 
                    frameon=True, 
                    framealpha=0,
                    edgecolor='w', 
                    fancybox=False, 
                    title_fontsize=TICK_SIZE)

    # Open axes
    for axis in ['bottom','left']:
        ax.spines[axis].set_linewidth(1.2)
        ax.spines[axis].set_color('k')
    for axis in ['top','right']:
        ax.spines[axis].set_linewidth(0)

    plt.tight_layout()


In [ ]:
# Figure 6: Estimated Marginal Means
lmm_accuracy_figure([emmeans_acc, emmeans_std])
for fmt in FIGTYPES:
    plt.savefig(os.path.join(folder_results, 'figure6_old.{:s}'.format(fmt)), bbox_inches='tight', dpi=300)


## Linear model across HMDs (Revision)

In [ ]:
%%R -i val -o em_acc_nohmd -o model_acc_nohmd

library('lme4')
library('emmeans')
library('sjstats')

formula <- acc_nomonoc ~ 1 + vision
model_acc_nohmd <- lm(data=val, formula)

# Model + Anova results
cat('* ANOVA Omnibus Results + Statistics: Accuracy (across HMDs)')
print(summary(model_acc_nohmd))
print(anova_stats(model_acc_nohmd))

# Post-Hoc Tests
cat('\n\n* Post-Hoc Comparisons: Accuracy (across HMDs)')
print(emmeans(model_acc_nohmd, pairwise ~ vision, adjust="holm")$contrasts)

# Export estimated marginal means (for Figure 6)
em_acc_nohmd <- summary(emmeans(model_acc_nohmd, specs = ~ 'vision'))


In [ ]:
%%R -i val -o em_sd_nohmd -o model_sd_nohmd

library('lme4')
library('emmeans')
library('sjstats')

formula <- sd_nomonoc ~ 1 + vision
model_sd_nohmd <- lm(data=val, formula)

# Model + Anova results
cat('* ANOVA Omnibus Results + Statistics: SD Precision')
print(summary(model_sd_nohmd))
print(anova_stats(model_sd_nohmd))

# Post-Hoc Tests
cat('\n\n* Post-Hoc Comparisons: SD Precision')
print(emmeans(model_sd_nohmd, pairwise ~ vision, adjust="holm")$contrasts)

# Export estimated marginal means (for Figure 6)
em_sd_nohmd <- summary(emmeans(model_sd_nohmd, specs = ~ 'vision'))


In [ ]:
# Add across-HMD model result to Figure 6 
emmeans_acc = {'HMD 1': {'mean': [em_acc.loc[(em_acc.vision == 'uncorrected') & (em_acc.hmd == 1), 'emmean'].values[0],
                            em_acc.loc[(em_acc.vision == 'contacts') & (em_acc.hmd == 1), 'emmean'].values[0],
                            em_acc.loc[(em_acc.vision == 'glasses') & (em_acc.hmd == 1), 'emmean'].values[0]], 
                         'sem':  [em_acc.loc[(em_acc.vision == 'uncorrected') & (em_acc.hmd == 1), 'SE'].values[0],
                            em_acc.loc[(em_acc.vision == 'contacts') & (em_acc.hmd == 1), 'SE'].values[0],
                            em_acc.loc[(em_acc.vision == 'glasses') & (em_acc.hmd == 1), 'SE'].values[0]]},
               'HMD 2': {'mean': [em_acc.loc[(em_acc.vision == 'uncorrected') & (em_acc.hmd == 2), 'emmean'].values[0],
                            em_acc.loc[(em_acc.vision == 'contacts') & (em_acc.hmd == 2), 'emmean'].values[0],
                            em_acc.loc[(em_acc.vision == 'glasses') & (em_acc.hmd == 2), 'emmean'].values[0]], 
                         'sem':  [em_acc.loc[(em_acc.vision == 'uncorrected') & (em_acc.hmd == 2), 'SE'].values[0],
                            em_acc.loc[(em_acc.vision == 'contacts') & (em_acc.hmd == 2), 'SE'].values[0],
                            em_acc.loc[(em_acc.vision == 'glasses') & (em_acc.hmd == 2), 'SE'].values[0]]},
              'Across HMDs': {'mean': [em_acc_nohmd.loc[em_acc_nohmd.vision == 'uncorrected', 'emmean'].values[0],
                             em_acc_nohmd.loc[em_acc_nohmd.vision == 'contacts', 'emmean'].values[0],
                             em_acc_nohmd.loc[em_acc_nohmd.vision == 'glasses', 'emmean'].values[0]], 
                        'sem':  [em_acc_nohmd.loc[em_acc_nohmd.vision == 'uncorrected', 'SE'].values[0],
                             em_acc_nohmd.loc[em_acc_nohmd.vision == 'contacts', 'SE'].values[0],
                                em_acc_nohmd.loc[em_acc_nohmd.vision == 'glasses', 'SE'].values[0]]} }

emmeans_std = {'HMD 1': {'mean': [em_sd.loc[(em_sd.vision == 'uncorrected') & (em_sd.hmd == 1), 'emmean'].values[0],
                            em_sd.loc[(em_sd.vision == 'contacts') & (em_sd.hmd == 1), 'emmean'].values[0],
                            em_sd.loc[(em_sd.vision == 'glasses') & (em_sd.hmd == 1), 'emmean'].values[0]], 
                         'sem':  [em_sd.loc[(em_sd.vision == 'uncorrected') & (em_sd.hmd == 1), 'SE'].values[0],
                            em_sd.loc[(em_sd.vision == 'contacts') & (em_sd.hmd == 1), 'SE'].values[0],
                            em_sd.loc[(em_sd.vision == 'glasses') & (em_sd.hmd == 1), 'SE'].values[0]]},
               
               'HMD 2': {'mean': [em_sd.loc[(em_sd.vision == 'uncorrected') & (em_sd.hmd == 2), 'emmean'].values[0],
                            em_sd.loc[(em_sd.vision == 'contacts') & (em_sd.hmd == 2), 'emmean'].values[0],
                            em_sd.loc[(em_sd.vision == 'glasses') & (em_sd.hmd == 2), 'emmean'].values[0]], 
                         'sem':  [em_sd.loc[(em_sd.vision == 'uncorrected') & (em_sd.hmd == 2), 'SE'].values[0],
                            em_sd.loc[(em_sd.vision == 'contacts') & (em_sd.hmd == 2), 'SE'].values[0],
                            em_sd.loc[(em_sd.vision == 'glasses') & (em_sd.hmd == 2), 'SE'].values[0]]},
              
                'Across HMDs': {'mean': [em_sd_nohmd.loc[em_sd_nohmd.vision == 'uncorrected', 'emmean'].values[0],
                             em_sd_nohmd.loc[em_sd_nohmd.vision == 'contacts', 'emmean'].values[0],
                             em_sd_nohmd.loc[em_sd_nohmd.vision == 'glasses', 'emmean'].values[0]], 
                    'sem':  [em_sd_nohmd.loc[em_sd_nohmd.vision == 'uncorrected', 'SE'].values[0],
                             em_sd_nohmd.loc[em_sd_nohmd.vision == 'contacts', 'SE'].values[0],
                             em_sd_nohmd.loc[em_sd_nohmd.vision == 'glasses', 'SE'].values[0]]}}


In [ ]:
# Figure 6: Estimated Marginal Means
lmm_accuracy_figure([emmeans_acc, emmeans_std])
for fmt in FIGTYPES:
    plt.savefig(os.path.join(folder_results, 'figure6.{:s}'.format(fmt)), bbox_inches='tight', dpi=300)


In [ ]:
%%R -i model_acc -i model_sd -i model_acc_nohmd -i model_sd_nohmd

# Model comparison
cat('* Model Comparison: Accuracy\n')
print(anova(model_acc_nohmd, model_acc))

cat('\n\n* Model Comparison: SD Precision\n')
print(anova(model_sd_nohmd, model_sd))